# Check time stamps and trianID

### Imports

In [ ]:
import h5py
import os.path
import struct
from datetime import datetime

### Define files

In [ ]:
file_path = '/home/cpassow/jupyterlab/CAMP/data/'
hdf_file_name = '20191031-060000_CH3I_fullFEL-test_merged.hdf5'
hdf_file_complete_path = file_path + hdf_file_name
assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'

live_file_name = '20191031-060000_CH3I_fullFEL_000000.flashid'
live_file_complete_path = file_path + live_file_name
assert os.path.isfile(live_file_complete_path), 'File does not exist!'

### From HDF file

In [ ]:
def data_from_hdf(hdf_file_complete_path):
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        time_stamps = h_file['timeStamps/ns'][:]
        ids = h_file['timeStamps/nr'][:]
    print('Reading from HDF5 - number of events: {}'.format(len(ids)))
    return time_stamps, ids

In [ ]:
time_stamps, ids = data_from_hdf(hdf_file_complete_path)

print(time_stamps[0])
print(time_stamps[1]-time_stamps[0])

print('First train:',datetime.utcfromtimestamp(int(time_stamps[0]*1E-9)).strftime('%Y-%m-%d %H:%M:%S'))
print('Last train: ',datetime.utcfromtimestamp(int(time_stamps[-1]*1E-9)).strftime('%Y-%m-%d %H:%M:%S'))

### Live trainID data from "Soeren code"
snippet for read_in from Hubertus

In [ ]:
size = struct.calcsize('<IIH')
ids_raw, seconds, usecs = [], [], []
with open(live_file_complete_path, 'rb') as f:
    while True:
        try:
            id, sec, usec = struct.unpack('<IIH', f.read(size))
            ids_raw.append(id)
            seconds.append(sec)
            usecs.append(usec)
        except:
            break
            
print('number of trainID entries in file:',len(ids_raw))

In [ ]:
ids_live = [ids_raw[i] for i in range(len(ids_raw)) if ids_raw[i]-ids_raw[i-1] != 0]
seconds_live =  [seconds[i] for i in range(len(ids_raw)) if ids_raw[i]-ids_raw[i-1] != 0]
usecs_live = [usecs[i] for i in range(len(ids_raw)) if ids_raw[i]-ids_raw[i-1] != 0]

print('number of trains after removing doubles:',len(ids_live))

In [ ]:
print('First train:',datetime.utcfromtimestamp(seconds_live[0]).strftime('%Y-%m-%d %H:%M:%S'),'|', ids_live[0])
print('Last train: ',datetime.utcfromtimestamp(seconds_live[-1]).strftime('%Y-%m-%d %H:%M:%S'),'|', ids_live[-1])             